# Merging the Spotify & Met cleaned databases

Combining the met & spotify databases, to make one recommender system

In [49]:
import numpy as np
import pandas as pd

df = pd.read_csv("TITLESSPELLCHECK14MAY.csv", header=None)
df

,0,1
0,NaN,0
1,0.0,Call on Me Ryan Back
2,1.0,You Make Me Feel Like A Natural Woman
3,2.0,Lush Life Is Now
4,3.0,None Of These Dollars in the Black
...,...,...
19888,19887.0,Rio the Second
19889,19888.0,I See Light In You Face
19890,19889.0,The One That I Like
19891,19890.0,Molly Child


In [50]:
df = df.drop(0)

In [51]:
df=df[1]
df

1                       Call on Me  Ryan Back 
2        You Make Me Feel Like A Natural Woman
3                             Lush Life Is Now
4           None Of These Dollars in the Black
5              You Really Got Int Woman Enough
                         ...                  
19888                           Rio the Second
19889                 I See Light In You  Face
19890                      The One That I Like
19891                             Molly Child 
19892                            Rio the Hush 
Name: 1, Length: 19892, dtype: object

In [52]:
df2 = pd.read_csv("TITLESSPELLCHECK9MAY.csv", header=None)
df2

,0,1
0,NaN,0
1,0.0,the Madonna and Child with the Infant
2,1.0,the Virgin and Child with the Infant
3,2.0,and Bravo Cigarettes
4,3.0,The Pirates Retreat from the Pirates of the
...,...,...
6239,6238.0,theWuther Demo
6240,6239.0,the Man with a Gun Theme
6241,6240.0,Prelude in Sharp
6242,6241.0,I Got a Neighbor


In [53]:
df2 = df2.drop(0)

In [54]:
df2=df2[1]
df2

1            the Madonna and Child with the Infant 
2             the Virgin and Child with the Infant 
3                              and Bravo Cigarettes
4       The Pirates Retreat from the Pirates of the
5            from the Actresses seriesissued by Kin
                           ...                     
6239                                 theWuther Demo
6240                       the Man with a Gun Theme
6241                              Prelude in Sharp 
6242                               I Got a Neighbor
6243              And This Is Still YouOriginal Mix
Name: 1, Length: 6243, dtype: object

In [55]:
titles = pd.concat(
    [df, df2],
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)

titles.describe()

count               26135
unique              26014
top       Do da Wordsmith
freq                    5
Name: 1, dtype: object

In [56]:
titles = titles.drop_duplicates()
titles = titles.reset_index(drop=True)
titles

0                       Call on Me  Ryan Back 
1        You Make Me Feel Like A Natural Woman
2                             Lush Life Is Now
3           None Of These Dollars in the Black
4              You Really Got Int Woman Enough
                         ...                  
26009                           theWuther Demo
26010                 the Man with a Gun Theme
26011                        Prelude in Sharp 
26012                         I Got a Neighbor
26013        And This Is Still YouOriginal Mix
Name: 1, Length: 26014, dtype: object

In [40]:
titles.to_csv("titlesfinal.csv")

# Creating the vector matrix

In [119]:
#https://www.datacamp.com/tutorial/recommender-systems-python
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(titles)

tfidf_matrix.shape

(26014, 10288)

In [117]:
tfidf.get_feature_names()[5000:5010]

['leaky',
 'leaning',
 'leap',
 'learn',
 'learning',
 'learnthee',
 'learnunplugged',
 'lease',
 'leather',
 'leatherette']

In [108]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [109]:
cosine_sim.shape

(26014, 26014)

In [110]:
indices = pd.Series(titles.index, index=titles).drop_duplicates()

# Creating thumbs up & thumbs down functions

In [131]:
#https://www.datacamp.com/tutorial/recommender-systems-python
def thumbs_up(title, cosine_sim=cosine_sim):
    
    index = indices[title]

    sim_scores = list(enumerate(cosine_sim[index]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[100:110] 

    prompt_indices = [i[0] for i in sim_scores]

    return titles.iloc[prompt_indices]

In [121]:
#https://www.datacamp.com/tutorial/recommender-systems-python
def thumbs_down(title, cosine_sim=cosine_sim):
    
    index = indices[title]

    sim_scores = list(enumerate(cosine_sim[index]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[5000:5010] #This is set pretty far away!!!

    prompt_indices = [i[0] for i in sim_scores]

    return titles.iloc[prompt_indices]

In [134]:
thumbs_up('The Night They Fight')

6063                     Space By Night O I
1646                         Boys Night Out
18165                            Boys Night
11355                    Grown Up All Night
3986        Loves See What The Night Can Do
8427               Stone The Night Together
745                        Rivers All Night
3500                   Rivers For The Night
14457    The Fight Song No One Ava Max Left
15295                           Carry Night
Name: 1, dtype: object

In [133]:
thumbs_down('The Last Dreaming Night')

4744     Reverse The Most Off  Throw
4745                     INN January
4746                    Tears On You
4747    I Long to Lady Jungle  El Ed
4748                      Station   
4749        Cirsonys Spaces To Prove
4750                 Tone it Dreamer
4751                Cherryapple Wine
4752                         Facts  
4753            Booming I Font  Sand
Name: 1, dtype: object

In [122]:
titles[2891]

'Poll Rouge'